In [1]:
import os
import pandas as pd
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
rootpath = os.path.dirname(os.path.abspath(""))
datapath = os.path.join(rootpath, 'data')
print(f"datapath: {datapath}")

datapath: /home/onur/repos/nytaxi-spark/data


In [3]:
spark = SparkSession.builder \
    .master('local[2]') \
    .appName('taxi_join') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 14:22:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read and Combine Green&Yellow Data

In [4]:
df_green = spark.read.parquet(os.path.join(datapath, 'report', 'revenue_hourly_green'))

In [13]:
df_green = df_green\
.withColumnRenamed('amount', 'green_amount')\
.withColumnRenamed('number_records', 'green_number_records')

In [5]:
df_green.count()

744767

In [14]:
df_green.columns

['hour', 'rzone', 'green_amount', 'green_number_records']

In [7]:
df_yellow = spark.read.parquet(os.path.join(datapath, 'report', 'revenue_hourly_yellow'))

In [15]:
df_yellow = df_yellow\
.withColumnRenamed('amount', 'yellow_amount')\
.withColumnRenamed('number_records', 'yellow_number_records')

In [8]:
df_yellow.count()

1520799

In [16]:
df_yellow.columns

['hour', 'rzone', 'yellow_amount', 'yellow_number_records']

In [17]:
 df_join = df_green.join(df_yellow, on=['hour', 'rzone'], how='outer')

In [18]:
df_join.show()

+-------------------+-----+------------------+--------------------+------------------+---------------------+
|               hour|rzone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+-----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|    3|              NULL|                NULL|              25.0|                    1|
|2020-01-01 00:00:00|    4|              NULL|                NULL|1004.3000000000002|                   57|
|2020-01-01 00:00:00|    7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|   12|              NULL|                NULL|             107.0|                    6|
|2020-01-01 00:00:00|   13|              NULL|                NULL|1214.8000000000002|                   56|
|2020-01-01 00:00:00|   18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:0

In [19]:
df_join.write.parquet(os.path.join(datapath, 'report', 'revenue_hourly_joined'))

### Joining large and small tables

In [20]:
df_join = spark.read.parquet(os.path.join(datapath, 'report', 'revenue_hourly_joined'))

In [21]:
df_join.show()

+-------------------+-----+------------------+--------------------+------------------+---------------------+
|               hour|rzone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+-----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|   14|              NULL|                NULL|               8.8|                    1|
|2020-01-01 00:00:00|   15|              NULL|                NULL|             34.09|                    1|
|2020-01-01 00:00:00|   17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|   22|              15.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|   24|              87.6|                   3| 754.9500000000002|                   45|
|2020-01-01 00:00:0

In [22]:
df_zones = spark.read.parquet(os.path.join(datapath, 'zones'))

In [23]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [24]:
df_result = df_join.join(df_zones, df_join.rzone == df_zones.LocationID)

In [29]:
df_result = df_result.drop('rzone', 'LocationID')

In [30]:
df_result.show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|              NULL|                NULL|             42.41|                    2|   Queens|        Baisley Park|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|               8.8|                    1| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|              NULL|                NULL|             34.09|                    1|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|195.03000000000003|                   9|220.20999999999998|                    8| Brooklyn|      

In [33]:
df_result.write.parquet(os.path.join(datapath, 'report', 'revenue_hourly_joined_zones'))

In [34]:
spark.sparkContext.stop()